In [2]:
from model import get_models_and_path
models = get_models_and_path()


<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>


In [7]:
resFreeze = models[-4]
print(resFreeze[0].__class__.__name__)
print(resFreeze[1])
res = models[-5]
print(res[0].__class__.__name__)
print(res[1])


Res18_1000
../remote_models/res18/RUBCNL_Res18_freeze/Res18_ep_29.pt
Res18_1000
../remote_models/res18/RUBCNL_Res18/Res18_ep_29.pt


In [14]:
target = "gene1.0.bias"

for name, param in resFreeze[0].named_parameters():
    if name != target:
        continue
    
    print(name, ": ", param.data)
    
for name, param in res[0].named_parameters():
    if name != target:
        continue
    #print(name)
    print(name, ": ", param.data)
    

gene1.0.bias :  tensor([-0.0157, -0.0120, -0.0284,  0.0181, -0.0187, -0.0394, -0.0364, -0.0172,
        -0.0017,  0.0069, -0.0075, -0.0240, -0.0262, -0.0044, -0.0277,  0.0123,
        -0.0167,  0.0159, -0.0265,  0.0084, -0.0421, -0.0064, -0.0145,  0.0108,
        -0.0102,  0.0271,  0.0006, -0.0163, -0.0268,  0.0197, -0.0011,  0.0061,
        -0.0068, -0.0176, -0.0076,  0.0208,  0.0230, -0.0279, -0.0147, -0.0316,
        -0.0195,  0.0273, -0.0061, -0.0021,  0.0154,  0.0300,  0.0041,  0.0015,
        -0.0242, -0.0033, -0.0202, -0.0095,  0.0183, -0.0366,  0.0024,  0.0041,
        -0.0084, -0.0318,  0.0138,  0.0041, -0.0123,  0.0111, -0.0102, -0.0511,
        -0.0046,  0.0163, -0.0274, -0.0294, -0.0433, -0.0336, -0.0085, -0.0301,
         0.0320, -0.0171, -0.0326, -0.0064, -0.0295, -0.0176, -0.0103,  0.0057,
         0.0071,  0.0107, -0.0131, -0.0260, -0.0371, -0.0060,  0.0035,  0.0022,
        -0.0345, -0.0018, -0.0297,  0.0050, -0.0080, -0.0213, -0.0509, -0.0450,
         0.0328, -0.0041